<a href="https://colab.research.google.com/github/apetros/pyeplan/blob/master/examples/5_bus_Example_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Investment and Operation Planning Module

This example considers a network of 5 nodes, with a load connected at each node. One representative day with three operational hours is used to describe the potential operational scenarios. Three types of investment candidate generators i.e, battery units, solar units and diesel generation units. We showcase how the Investment and Operation Planning Modules of PyEPLAN can be used to derive optimal units to be installed in the network.

This example is designed to be run on Google Colab.

In [ ]:
!rm -r sample_data
!rm -r wat_inv

In [ ]:
!apt-get install -qq subversion
!svn export https://github.com/apetros/pyeplan/trunk/examples/wat_inv

In [ ]:
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc

In [ ]:
!pip install -q pyeplan

In [ ]:
!pip install git+git://github.com/apetros/pyeplan@master

## Importing the invesment and operation planning module from PyEPLAN 

In [ ]:
from pyeplan import inosys

## Defining the input data

The module is initialised with inputs including but not limited the following :
* The input directory for the data. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).
* ref_bus: Reference node
* dshed_cost: Demand Shedding Price
* rshed_cost: Renewable Shedding Price
* phase: Number of Phases (Default 3)
* sbase: Base Apparent Power (Default 1 kW)

In [ ]:
inosys = inosys('5_bus', ref_bus = 0, dshed_cost = 100, rshed_cost = 0, phase = 3,)

Some of the data inputs include:

### Defining the total load demand at each hour

The total active power consumption at each of the three (3) hours for one (1) representative day is:

In [ ]:
import pandas as pd
pd.read_csv("5_bus/prep_dist.csv")

,0
0,24.6
1,55.4
2,80.7


### Defining the input investment candidates

#### Battery Units

In [ ]:
pd.read_csv("5_bus/cbat_dist.csv")

,bus,icost,ocost,scost,ec,ed,emax,emin,eini,pmax,pmin,qmax,qmin
0,0,347.2176,0,0,0.92,0.92,1200,0,600,100,0,0,0
1,4,347.2176,0,0,0.92,0.92,1200,0,600,100,0,0,0


#### Solar PV Units

In [ ]:
pd.read_csv("5_bus/csol_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,108.766,0,0,0,50,-50,50
1,4,108.766,0,0,0,50,-50,50


#### Diesel/Fossil Units

In [ ]:
pd.read_csv("5_bus/cgen_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax,hr
0,0,12.0345,0.27,0,0,100,0,0,3


## Solving the optimization problem

PyEPLAN can be used to solve the problem investment and operation planning problems simultaneously. In case no investment candidates are availble, a sole operation planning problem can be run by setting input 'onlyopr = True'. Available solvers inclde both open source solvers include glpk, cbc, and commercial solvers ipopt, gurobi given one has the required licences. If discrete capacities of investment units are available the input 'invest = True' this sets the investement-related decision variables to a binary nature where the unit capacity = maximum capacity. Otherwise,  'invest = False' sets the investement-related decision variables to a continuous nature where the unit capacity <= maximum capacity.

In [ ]:
inosys.solve(solver = 'glpk', onlyopr = False, invest = False, )

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\BLESSI~1\AppData\Local\Temp\tmp464szbjy.glpk.raw --wglp
 C:\Users\BLESSI~1\AppData\Local\Temp\tmp9v8mmyme.glpk.glp --cpxlp C:\Users\BLESSI~1\AppData\Local\Temp\tmp2aszm9s4.pyomo.lp
Reading problem data from 'C:\Users\BLESSI~1\AppData\Local\Temp\tmp2aszm9s4.pyomo.lp'...
218 rows, 133 columns, 502 non-zeros
1297 lines were read
Writing problem data to 'C:\Users\BLESSI~1\AppData\Local\Temp\tmp9v8mmyme.glpk.glp'...
1087 lines were written
GLPK Simplex Optimizer 5.0
218 rows, 133 columns, 502 non-zeros
Preprocessing...
99 rows, 74 columns, 289 non-zeros
Scaling...
 A: min|aij| =  1.852e-08  max|aij| =  6.000e+02  ratio =  3.241e+10
GM: min|aij| =  5.275e-01  max|aij| =  1.896e+00  ratio =  3.594e+00
EQ: min|aij| =  2.887e-01  max|aij| =  1.000e+00  ratio =  3.463e+00
Constructing initial basis...
Size of triangular part is 99
      0: obj =   5.865550000e+06 inf =   9.377e+05 (21)
     34: obj =   5

## Results

A folder named 'results' will be created with the output of the optimal solution to the planning problem. The different result files are defined [here](https://pyeplan.sps-lab.org/user_guide/output.html). Below we show the capital costs and operational costs obtained to satify the load in the 5-bus network.

### Total investment and operational costs

In [ ]:
pd.read_csv("5_bus/results/obj.csv")

,total costs,14055.479202
0,total investment costs,11125.319656
1,total operation costs,2930.159546


### Number and capacity of battery units installed

The capacity and location of battery units installed is:

In [ ]:
import numpy as np
cbat = pd.read_csv("5_bus/cbat_dist.csv")
ibat = pd.read_csv("5_bus/results/xb.csv")
cbat['Unit'] = (np.arange(1,len(ibat.columns)+1))
unit = cbat.loc[:,'Unit']
bus = np.array(cbat.loc[:,'bus'])
out_bat =(((cbat.loc[:,'pmax']*round(ibat.loc[0:,].T,2))[0]).to_frame().set_index(unit)).rename(columns={0: 'Installed Capacity (kW)'})
out_bat['Bus'] = bus
out_bat.style

,Installed Capacity (kW),Bus
Unit,,
1,0.000000,0
2,0.000000,4


### Number and capacity of solar units installed

The capacity and location of solar units installed is:

In [ ]:
csol = pd.read_csv("1_bus/csol_dist.csv")
isol = pd.read_csv("1_bus/results/xs.csv")
csol['Unit'] = (np.arange(1,len(isol.columns)+1))
unit = csol.loc[:,'Unit']
bus = np.array(csol.loc[:,'bus'])
out_sol =(((csol.loc[:,'pmax']*round(isol.loc[0:,].T,2))[0]).to_frame().set_index(unit)).rename(columns={0: 'Installed Capacity (kW)'})
out_sol['Bus'] = bus
out_sol.style

,Installed Capacity (kW),Bus
Unit,,
1,50.000000,0
2,50.000000,0


### Number and capacity of diesel units installed

The capacity and location of diesel units installed is:

In [ ]:
cgen = pd.read_csv("5_bus/cgen_dist.csv")
igen = pd.read_csv("5_bus/results/xg.csv")
bus = cgen.loc[:,'bus']
(((cgen.loc[:,'pmax']*round(igen.loc[0:,].T,2))[0]).to_frame().set_index(bus)).rename(columns={0: 'Installed Capacity (kW)'})

,Installed Capacity (kW)
bus,
0,21.0


### Amount of load curtailed

The level of load demand that has been curtailed at each node is:

In [ ]:
pds =pd.read_csv("5_bus/results/pds.csv")
pds.index.name ='Hour'
pds.style

,0,1,2,3,4
Hour,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000


## Remarks

This example showcases the planning modules in PyEPLAN. Different solutions can be obtained by varrying the input data based on the user's own system data.